## 1. Importaciones y Configuración

Importamos las bibliotecas necesarias:
- **FunctionInvocationContext**: Proporciona acceso al contexto de la llamada a función
- **Callable y Awaitable**: Para type hints del middleware
- **AzureAIAgentClient**: Cliente del servicio Azure AI Agent

In [ ]:
import os
import asyncio
from typing import Awaitable, Callable
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from agent_framework import FunctionInvocationContext
from dotenv import load_dotenv

load_dotenv()

print("✅ Importaciones completadas")

## 2. Definición de la Función Tool

### La Función `get_time`

Una función simple que retorna la hora actual. Esta función será llamada por el agente cuando el usuario pregunte por la hora.

#### Características:
- No requiere parámetros
- Usa `datetime` para obtener la hora actual
- Retorna un string formateado como "HH:MM:SS"
- El docstring ayuda al LLM a entender su propósito

In [ ]:
def get_time():
    """Obtiene la hora actual."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

print("✅ Función get_time definida")

# Probar la función
print(f"\n🕐 Hora actual: {get_time()}")

## 3. Implementación del Middleware de Logging

### ¿Qué es un Middleware?

Un middleware es una función que:
1. Se ejecuta **antes** de la operación principal (pre-processing)
2. Permite que la operación se ejecute llamando a `next(context)`
3. Se ejecuta **después** de la operación (post-processing)

### Estructura del Middleware:

```python
async def middleware(context, next):
    # PRE: Código antes de la ejecución
    print(f"Calling: {context.function.name}")
    
    # EJECUCIÓN: Llamar a la siguiente función en la cadena
    await next(context)
    
    # POST: Código después de la ejecución
    print(f"Result: {context.result}")
```

### FunctionInvocationContext:

Este objeto proporciona acceso a:
- **context.function**: Información sobre la función (nombre, parámetros, docstring)
- **context.result**: Resultado de la ejecución de la función
- **context.arguments**: Argumentos pasados a la función
- **context.exception**: Excepción si ocurrió un error

### Casos de Uso del Middleware:
1. **Logging y Auditoría**: Registrar todas las llamadas a funciones
2. **Performance Monitoring**: Medir tiempo de ejecución
3. **Validación**: Verificar argumentos antes de la ejecución
4. **Autorización**: Verificar permisos
5. **Rate Limiting**: Controlar frecuencia de llamadas
6. **Caché**: Evitar llamadas repetidas
7. **Error Handling**: Manejo centralizado de errores

In [ ]:
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware que registra llamadas a funciones."""
    
    # PRE-EJECUCIÓN: Registrar que se va a llamar a la función
    print(f"🔵 [PRE] Calling function: {context.function.name}")

    # EJECUCIÓN: Continuar con la siguiente función en la cadena
    # Esta línea ejecuta la función real (get_time en nuestro caso)
    await next(context)

    # POST-EJECUCIÓN: Registrar el resultado
    print(f"🟢 [POST] Function result: {context.result}")

print("✅ Middleware de logging definido")

## 4. Creación y Ejecución del Agente con Middleware

### Configuración del Agente:

El agente se configura con:
1. **name**: Identificador del agente
2. **instructions**: Comportamiento del agente
3. **tools**: Lista de funciones disponibles (get_time)
4. **middleware**: Lista de middleware a aplicar

### Flujo de Ejecución:

```
1. Usuario: "What time is it?"
2. Agente analiza la consulta
3. Agente decide llamar a get_time()
4. Middleware PRE: Log "Calling function: get_time"
5. Función get_time() se ejecuta → "14:23:45"
6. Middleware POST: Log "Function result: 14:23:45"
7. Agente formula respuesta: "The current time is 14:23:45"
8. Usuario recibe respuesta
```

### Context Manager:

Usamos `async with` para:
- Gestión automática de recursos
- Cierre apropiado de conexiones
- Limpieza en caso de errores

In [ ]:
async def main():
    print("🚀 Iniciando agente con middleware de logging...\n")
    
    credential = AzureCliCredential()

    async with AzureAIAgentClient(
        async_credential=credential, 
        project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
        model_deployment_name=os.getenv("MODEL")
    ).create_agent(
        name="TimeAgent",
        instructions="Puedes decir la hora actual.",
        tools=[get_time],
        middleware=[logging_function_middleware],  # ⭐ Agregar middleware aquí
    ) as agent:
        print("❓ Pregunta: What time is it?\n")
        print("📋 Logs del middleware:")
        print("-" * 60)
        
        result = await agent.run("What time is it?")
        
        print("-" * 60)
        print("\n💬 Respuesta del agente:")
        print("=" * 60)
        print(result.text)
        print("=" * 60)

print("✅ Función main definida")

## 5. Ejecución del Agente

Ejecutamos el agente y observamos los logs del middleware en acción:

In [ ]:
await main()

## 6. Middleware Avanzado: Performance Monitoring

Ejemplo de un middleware más sofisticado que mide el tiempo de ejecución:

In [ ]:
import time

async def performance_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware que mide el tiempo de ejecución de funciones."""
    
    # Registrar tiempo de inicio
    start_time = time.perf_counter()
    print(f"⏱️  [START] Executing: {context.function.name}")
    
    # Ejecutar la función
    await next(context)
    
    # Calcular tiempo transcurrido
    elapsed = time.perf_counter() - start_time
    print(f"⏱️  [END] {context.function.name} took {elapsed*1000:.2f}ms")

print("✅ Middleware de performance definido")

## 7. Composición de Múltiples Middlewares

Podemos aplicar múltiples middlewares en cadena. Se ejecutan en orden:

In [ ]:
async def test_multiple_middlewares():
    print("🧪 Probando múltiples middlewares en cadena...\n")
    
    credential = AzureCliCredential()

    async with AzureAIAgentClient(
        async_credential=credential, 
        project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
        model_deployment_name=os.getenv("MODEL")
    ).create_agent(
        name="TimeAgent",
        instructions="Puedes decir la hora actual.",
        tools=[get_time],
        middleware=[
            logging_function_middleware,    # Primero: logging
            performance_middleware,         # Segundo: performance
        ],
    ) as agent:
        print("📋 Observa el orden de ejecución de los middlewares:\n")
        result = await agent.run("What time is it?")
        print(f"\n💬 Respuesta: {result.text}")

await test_multiple_middlewares()

## 8. Middleware con Manejo de Errores

Ejemplo de middleware que captura y maneja errores:

In [ ]:
async def error_handling_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware que maneja errores en llamadas a funciones."""
    
    try:
        print(f"🛡️  [GUARD] Protecting call to: {context.function.name}")
        await next(context)
        print(f"🛡️  [GUARD] Successful execution")
    except Exception as e:
        print(f"❌ [ERROR] Function {context.function.name} failed: {str(e)}")
        # Aquí podrías:
        # - Registrar el error en un sistema de logging
        # - Enviar alertas
        # - Intentar recuperación
        # - Retornar un valor por defecto
        raise  # Re-lanzar la excepción

print("✅ Middleware de error handling definido")

## 9. Análisis y Conclusiones

### Ventajas del Patrón Middleware:

1. **Separación de Responsabilidades**:
   - La lógica de negocio (get_time) está separada del logging
   - Cada middleware tiene un propósito único
   - Código más limpio y mantenible

2. **Reutilización**:
   - El mismo middleware puede usarse con diferentes funciones
   - No es necesario modificar las funciones originales
   - Composición flexible

3. **Extensibilidad**:
   - Fácil agregar nueva funcionalidad sin modificar código existente
   - Principio Open/Closed de SOLID
   - Arquitectura plug-and-play

4. **Testing**:
   - Middleware y funciones pueden testearse independientemente
   - Mocking más sencillo
   - Mejor cobertura de tests

### Casos de Uso Prácticos:

1. **Auditoría y Compliance**:
   - Registrar todas las operaciones para auditorías
   - Tracking de quién ejecutó qué y cuándo
   - Cumplimiento de regulaciones (GDPR, HIPAA, etc.)

2. **Debugging y Troubleshooting**:
   - Logs detallados de ejecución
   - Trazabilidad de problemas en producción
   - Reproducción de errores

3. **Performance Optimization**:
   - Identificar funciones lentas
   - Optimización basada en métricas reales
   - Alertas de performance

4. **Seguridad**:
   - Validación de inputs
   - Control de acceso
   - Detección de patrones sospechosos

5. **Caché y Optimización**:
   - Caché de resultados frecuentes
   - Evitar llamadas redundantes
   - Mejora de throughput

### Patrones Avanzados:

1. **Conditional Middleware**:
   ```python
   if context.function.name in ["critical_function"]:
       # Aplicar lógica especial
   ```

2. **Context Enhancement**:
   ```python
   context.custom_data = {"user_id": get_current_user()}
   await next(context)
   ```

3. **Result Transformation**:
   ```python
   await next(context)
   context.result = transform(context.result)
   ```

### Mejores Prácticas:

1. **Keep It Simple**: Cada middleware debe hacer una sola cosa
2. **Orden Importa**: Considera el orden de ejecución de middlewares
3. **Performance**: Middleware debe ser eficiente para no impactar latencia
4. **Error Handling**: Siempre manejar errores apropiadamente
5. **Documentation**: Documentar el propósito y comportamiento de cada middleware

### Extensiones Posibles:

1. **Structured Logging**: Usar bibliotecas como structlog para logs estructurados
2. **Metrics Export**: Enviar métricas a Prometheus/Grafana
3. **Distributed Tracing**: Integrar con OpenTelemetry
4. **Rate Limiting**: Implementar throttling de llamadas
5. **Circuit Breaker**: Proteger contra fallos cascada